In [1]:
import os 
import numpy as np
from PIL import Image 
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

2023-05-09 06:26:17.863713: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-09 06:26:17.917502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 06:26:18.846187: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/envs/cs109b/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as noteboo

In [2]:
tf.config.run_functions_eagerly(True)

In [3]:
div2k_data = tfds.image.Div2k(config="bicubic_x4")
div2k_data.download_and_prepare()

# # Taking train data from div2k_data object
div2k_train = div2k_data.as_dataset(split="train", as_supervised=True)
div2k_val = div2k_data.as_dataset(split="validation", as_supervised=True)

2023-05-09 06:26:36.789788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 06:26:36.820622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 06:26:36.820986: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [1]:
def sample(ds: tf.data.Dataset, mode='large'):
    """
    This helper function resamples patches from the original training image. It takes in the tf.Dataset object 
    and returns two numpy arrays
    
    params:
    ds :: tf.data.Dataset :: TensorFlow Dataset, by default is the Div2K training set.
    mode (optional) :: string :: Specify whether large patches (224x224) or small patches (96x96) are needed, default to large.
    """
    
    if mode == 'large':
        stride, size = 224, 224
    elif mode == 'small':
        stride, size = 360, 96
    else:
        print("Please specify your mode as 'large' (224x224 patches) or 'small' (96x96 patches)")
    low, high = [], []
    for lr, hr in ds.as_numpy_iterator():
        h, w, c = hr.shape
        nh, nw = (h-size) // stride, (w-size) // stride
        xh, yh, xl, yl = 0, 0, 0, 0
        for i in range(nh):
            for j in range(nw):
                patch = hr[xh:xh+size, yh:yh+size, :]
                patch_small  = lr[xl:xl+size//4, yl:yl+size//4, :]
                assert patch.shape == (size, size, 3) and patch_small.shape == (size//4, size//4, 3), print(xh, yh,hr.shape, patch.shape)
                low.append(patch_small)
                high.append(patch)
                yh += stride
                yl += stride // 4
            xh += stride
            xl += stride // 4
            yl, yh = 0, 0
    return np.stack(low), np.stack(high)
    

NameError: name 'tf' is not defined

In [5]:
# Resample training dataset and convert to tf.data.Dataset objects
low_dev, high_dev = sample(div2k_train)
ds_dev = tf.data.Dataset.from_tensor_slices((low_dev, high_dev))

2023-05-09 06:27:24.899494: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [32]
	 [[{{node Placeholder/_4}}]]
2023-05-09 06:27:24.899982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [32]
	 [[{{node Placeholder/_2}}]]


In [7]:
# Train-validation split and cache the Dataset
train_size = ds_dev.cardinality()*85//100
ds_train = ds_dev.take(train_size)
ds_val = ds_dev.skip(train_size)
ds_train.save('dataset/train') # This is the default path, you may change to other path
ds_val.save('dataset/val') # This is the default path, you may change to other path

2023-05-09 06:29:10.358413: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [12245,96,96,3]
	 [[{{node Placeholder/_1}}]]
2023-05-09 06:29:11.689819: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [12245,96,96,3]
	 [[{{node Placeholder/_1}}]]


In [8]:
ds_train.cardinality()

<tf.Tensor: shape=(), dtype=int64, numpy=12933>

In [9]:
div2k_val.save('dataset/test/')

/opt/conda/envs/cs109b/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2023-05-08 17:41:31.520692: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [4]
	 [[{{node Placeholder/_0}}]]
2023-05-08 17:41:31.521102: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int